In [2]:
import xml.etree.ElementTree as ET
import pygraphviz as pgv

def extract_sequence(xml_code):
    """
    Extrai a sequência de execução a partir de um bloco de código XML de um workflow Pentaho.
    
    Args:
        xml_code (str): Bloco de código XML.
    
    Returns:
        list: Lista de tuplas representando as conexões (from, to).
    """
    root = ET.fromstring(xml_code)
    hops = []
    
    for hop in root.findall('hop'):
        from_step = hop.find('from').text
        to_step = hop.find('to').text
        enabled = hop.find('enabled').text
        
        if enabled == 'Y':
            hops.append((from_step, to_step))
    
    return hops

def create_flow_graph(hops, output_file="flow_graph.png"):
    """
    Cria um fluxo de dados usando pygraphviz e salva a imagem gerada.
    
    Args:
        hops (list): Lista de tuplas representando as conexões (from, to).
        output_file (str): Nome do arquivo de saída para salvar o gráfico.
    """
    graph = pgv.AGraph(directed=True)
    
    for from_step, to_step in hops:
        graph.add_edge(from_step, to_step)
    
    graph.layout(prog='dot')
    graph.draw(output_file)
    print(f"Fluxo de dados salvo em {output_file}")

# Exemplo de uso
xml_code = """
  <order>
  <hop> <from>Switch &#47; Case LOCAL FATURAMENTO</from><to>Switch &#47; Case TITULAR RESPONSÁVEL</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case PESSOA RESPONSÁVEL</from><to>Dummy (do nothing) 2</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case PESSOA RESPONSÁVEL</from><to>FAMILIA PESSOA RESPONSÁVEL</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case TITULAR RESPONSÁVEL</from><to>FAMILIA TITULAR RESPONSÁVEL</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case TITULAR RESPONSÁVEL</from><to>Switch &#47; Case PESSOA RESPONSÁVEL</to><enabled>Y</enabled> </hop>  
  <hop> <from>Dummy (do nothing)</from><to>Remover colunas</to><enabled>Y</enabled> </hop>  
  <hop> <from>Remover colunas</from><to>Switch &#47; Case LOCAL FATURAMENTO</to><enabled>Y</enabled> </hop>  
  <hop> <from>BENEFICIÁRIO</from><to>SAM_FAMILIA_TETO_PF</to><enabled>Y</enabled> </hop>  
  <hop> <from>SAM_FAMILIA_TETO_PF</from><to>Dummy (do nothing)</to><enabled>Y</enabled> </hop>  
  <hop> <from>SEM SETOR</from><to>Insert &#47; Update - BN_BENEFICIARIO</to><enabled>Y</enabled> </hop>  
  <hop> <from>Insert &#47; Update - BN_BENEFICIARIO</from><to>QTD_INCATU_BN_BENEFICIARIO</to><enabled>Y</enabled> </hop>  
  <hop> <from>Filter rows</from><to>BUSCA MICROSIGA</to><enabled>Y</enabled> </hop>  
  <hop> <from>BUSCA MICROSIGA</from><to>Insert &#47; Update - BN_BENEFICIARIO</to><enabled>Y</enabled> </hop>  
  <hop> <from>HANDLE_BENEFICIARIO</from><to>BENEFICIÁRIO</to><enabled>Y</enabled> </hop>  
  <hop> <from>Filter rows</from><to>SEM SETOR</to><enabled>Y</enabled> </hop>  
  <hop> <from>Blocking Step</from><to>Set Variables</to><enabled>Y</enabled> </hop>  
  <hop> <from>QTD_INCATU_BN_BENEFICIARIO</from><to>Blocking Step</to><enabled>Y</enabled> </hop>  
  <hop> <from>Dummy (do nothing)</from><to>Filter rows</to><enabled>Y</enabled> </hop>  
  <hop> <from>Blocking Step 2</from><to>Set Variables 2</to><enabled>Y</enabled> </hop>  
  <hop> <from>CONTRATO</from><to>Insert &#47; Update - BN_RESP_FINANCEIRO</to><enabled>Y</enabled> </hop>  
  <hop> <from>FAMILIA PESSOA RESPONSÁVEL</from><to>Insert &#47; Update - BN_RESP_FINANCEIRO</to><enabled>Y</enabled> </hop>  
  <hop> <from>FAMILIA TITULAR RESPONSÁVEL</from><to>Insert &#47; Update - BN_RESP_FINANCEIRO</to><enabled>Y</enabled> </hop>  
  <hop> <from>Insert &#47; Update - BN_RESP_FINANCEIRO</from><to>QTD_INCATU_BN_RESP_FINANCEIRO</to><enabled>Y</enabled> </hop>  
  <hop> <from>LOTAÇÃO</from><to>Insert &#47; Update - BN_RESP_FINANCEIRO</to><enabled>Y</enabled> </hop>  
  <hop> <from>QTD_INCATU_BN_RESP_FINANCEIRO</from><to>Blocking Step 2</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case LOCAL FATURAMENTO</from><to>CONTRATO</to><enabled>Y</enabled> </hop>  
  <hop> <from>Switch &#47; Case LOCAL FATURAMENTO</from><to>LOTAÇÃO</to><enabled>Y</enabled> </hop>  
  </order>
"""

# Extraindo a sequência e criando o fluxo de dados
hops = extract_sequence(xml_code)
create_flow_graph(hops)

Fluxo de dados salvo em flow_graph.png


In [3]:
import xml.etree.ElementTree as ET
import logging

def parse_ktr_file(file):
    try:
        tree = ET.parse(state["file_path"])
        state["root"] = tree.getroot()
    except ET.ParseError as e:
        logging.error(f"Error parsing KTR file: {e}")
        raise
    return state

def parse_insert_update_step(xml_string):
    root = ET.fromstring(xml_string)
    
    # Encontrar o step "Insert / Update"
    step = root.find(".//step[name='Insert / Update - BN_RESP_FINANCEIRO']")
    
    if step is None:
        print("Step 'Insert / Update - BN_RESP_FINANCEIRO' não encontrado.")
        return
    
    # Extrair informações básicas
    table = step.find(".//table").text
    print(f"Tabela: {table}")
    
    # Extrair informações sobre as chaves
    print("\nChaves:")
    for key in step.findall(".//key"):
        name = key.find("name").text
        field = key.find("field").text
        print(f"  Nome: {name}, Campo: {field}")
    
    # Extrair informações sobre os campos
    print("\nCampos:")
    for value in step.findall(".//value"):
        name = value.find("name").text
        rename = value.find("rename").text
        update = value.find("update").text
        
        if name != rename:
            print(f"  Nome: {name}, Renomeado para: {rename}, Atualizar: {update}")
        else:
            print(f"  Nome: {name}, Atualizar: {update}")



In [ ]:
# Uso da função
xml_string = '''
<step>
    <name>Insert / Update - BN_RESP_FINANCEIRO</name>
    <!-- ... resto do XML ... -->
</step>
'''

parse_insert_update_step(xml_string)